In [1]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer, WhisperConfig
from sklearn.feature_extraction.text import CountVectorizer

config = WhisperConfig.from_pretrained('openai/whisper-large-v3')
maxlen = config.max_length - 3

In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from datasets import Audio

sr = 16000
audio = Audio(sampling_rate=sr)

In [4]:
files = sorted(glob('output-indonesian-v2/*.json'), key = lambda x: int(x.split('-')[-1].replace('.json', '')))
len(files)

10328

In [5]:
import re

pattern_pair = r'<\|(\d+\.\d+)\|>(.*?)<\|(\d+\.\d+)\|>'
matches = re.findall(pattern_pair, '<|0.00|> kerajaan persekutuan<|1.46|><|1.46|> dan banyak masalah hubungan<|3.96|><|3.96|> antara kerajaan negeri dan')
matches

[('0.00', ' kerajaan persekutuan', '1.46'),
 ('1.46', ' dan banyak masalah hubungan', '3.96')]

In [6]:
import string

punct = set(string.punctuation)

def remove_punct(s):
    return ''.join([c for c in s if c not in punct])

def remove_duplicate(string, n = 3):
    splitted = string.split()
    n = [splitted[i: i + n] for i in range(0, len(splitted), n)]
    already = set()
    dedup = []
    for n_ in n:
        original_n = ' '.join(n_)
        n_ = ' '.join(n_).lower()
        n_ = remove_punct(n_)
        if n_ not in already:
            dedup.append(original_n)
            already.add(n_)
    return ' '.join(dedup)

In [7]:
import math

def round_to_nearest_0_02(number):
    return round(number * 50) / 50

In [8]:
selected = [
    'terima kasih kerana menonton',
    'terima kasih',
]

In [9]:
# with open(files[0]) as fopen:
#     d = json.load(fopen)
    
# d[0]

In [10]:
# tokenizer.decode(d[0]['predict_en'], decode_with_timestamps = True)

In [11]:
from huggingface_hub import hf_hub_download

In [12]:
f = hf_hub_download(
    repo_id="mesolitica/pseudolabel-indonesian-large-v3-timestamp", 
    filename="translated-indonesian-ms.json",
    repo_type = 'dataset'
)

In [13]:
with open(f) as fopen:
    translation = json.load(fopen)

len(translation)

330524

In [14]:
keys = translation.keys()

In [15]:
popped = []
for k in list(keys):
    v = translation[k]
    v = re.sub(r'\s-', '-', v)
    v = v.replace('< >', '<>').replace(' >', '<>')
        
    translation[k] = v
    k_split = k.split('<>')
    v_split = v.split('<>')
    
    if len(k_split) != len(v_split):
        translation.pop(k, None)
        popped.append((k_split, v_split, k, v))

In [16]:
len(translation)

305534

In [17]:
len(popped)

24990

In [18]:
import mp
import copy

minimum_score = 5

def loop(files):
    
    tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')
    
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            
            audio_filename = data[i]['filename']
            if not os.path.exists(audio_filename):
                continue
                
            y = audio.decode_example(audio.encode_example(audio_filename))['array']
            len_y = len(y) / sr
            if len_y > 30:
                continue
                
            rounded_num = f'<|{round_to_nearest_0_02(len_y):.2f}|>'
                    
            
            if data[i]['score_id'] > minimum_score:
                
                a = tokenizer._decode_asr(
                [{'tokens': np.array([data[i]['predict_id']])}], 
                return_timestamps = True, return_language = 'id', 
                time_precision = 0.02)[1]['chunks']
            
                a = [a_['text'] for a_ in a]
                
                t = ' <>'.join(a)
                if t not in translation:
                    continue
                    
                t_translated = translation[t]
                t_translated_splitted = t_translated.split('<>')
                if len(t_translated_splitted) != len(a):
                    continue
                
                mapping = {}
                for k in range(len(a)):
                    s = t_translated_splitted[k].strip()
                    if a[k][0] == ' ':
                        s = ' ' + s
                    if a[k][-1] == ' ':
                        s = s + ' '
                    mapping[a[k]] = s
            
                a = np.array(data[i]['predict_id'])
                a = a[a != 50257].tolist() + [50257]
                t = tokenizer.decode(a, skip_special_tokens = True, decode_with_timestamps = True).strip()
                if t.split('|>')[-1] != '':
                    t += rounded_num
                
                matches = re.findall(pattern_pair, t)
                rs = []
                for match in matches:
                    l = float(match[0])
                    r = float(match[2])
                    t_ = match[1]
                    rt_ = re.sub('[^a-z ]+', '', t_.lower()).strip()
                    
                    if (r - l > 3) and any([s == rt_ for s in selected]):
                        # print(audio_filename, t_)
                        t_ = ''
                    else:
                        if t_ in mapping:
                            t_ = mapping[t_]
                        else:
                            t_ = ''
                    
                    splitted = t_.split()
                    if len(splitted):
                        ratio = (len(set(splitted)) / len(splitted))
                        if len(t_) > 100 and ratio < 0.5:
                            t_ = remove_duplicate(t_)
                    if len(t_) and t_[0] != ' ':
                        t_ = ' ' + t_
                    rs.append(f'<|{match[0]}|>{t_}<|{match[2]}|>')
                rs = ''.join(rs)
                t = f'<|startoftranscript|><|ms|><|transcribe|>{rs}<|endoftext|>'
                d = {
                    'new_text': t,
                    'audio_filename': audio_filename,
                }
                results.append(d)
    return results

In [19]:
results = loop((files[:100], 0))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
 29%|██▉       | 29/100 [00:29<00:57,  1.24it/s]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
 90%|█████████ | 90/100 [12:40<13:09, 78.92s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
100%|██████████| 100/100 [19:56<00:00, 11.96s/it]


In [20]:
len(results)

3137

In [21]:
results[2]

{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> kenapa? Sebab konsepnya macam ni kak.<|1.50|><|1.60|> Abang, contohnya, tinggi dia 160.<|3.62|><|4.46|> Tapi abang, rasa macam nak dipandang tinggi oleh orang lain.<|7.16|><|7.48|> Jadi, sebagai contoh, orang yang lebih tinggi daripada anda,<|9.96|><|10.10|> ia seperti anda menekan mereka ke bawah supaya mereka turun.<|12.50|><|12.90|> Cuma supaya awak lebih tinggi daripada dia, abang masih 160.<|15.90|><|16.78|> Well, point yang saya nak sampaikan ialah,<|18.46|><|18.92|> kita ada kecenderungan setiap hari.<|20.50|><|20.72|> Nah, itu dipanggil kesilapan seorang lelaki.<|22.26|><|22.26|> Jadi sudah menjadi kebiasaan, kesilapan manusia yang sering kita lakukan.<|26.30|><|26.72|> Jadi kita mahu mengangkat diri kita dengan menghancurkan orang lain.<|29.86|><|endoftext|>',
 'audio_filename': 'split-indonesian/0-1-45120000.mp3'}

In [22]:
# [r for r in results if r['new_text'].count('<|') > 6]

In [23]:
results[100]['new_text'].count('<|')

34

In [24]:
import IPython.display as ipd
ipd.Audio('split-indonesian/0-137-126240000.mp3')

In [25]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-medium')
ori_tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
tokenizer.tokenize(results[-1]['new_text'])

['<|startoftranscript|>',
 '<|ms|>',
 '<|transcribe|>',
 '<|0.00|>',
 'ĠJadi',
 'Ġakhir',
 'Ġsekali',
 ',',
 'Ġkami',
 'Ġmem',
 'oh',
 'on',
 'Ġkepada',
 'Ġk',
 'era',
 'ja',
 'an',
 'Ġdan',
 'Ġal',
 'ham',
 'dul',
 'illah',
 ',',
 'ĠM',
 'enter',
 'i',
 'ĠAg',
 'ama',
 'Ġjuga',
 'Ġtel',
 'ah',
 'Ġmer',
 'ay',
 'u',
 'Ġkepada',
 'Ġk',
 'era',
 'ja',
 'an',
 '-',
 'k',
 'era',
 'ja',
 'an',
 'Ġda',
 'er',
 'ah',
 'Ġag',
 'ar',
 'Ġterus',
 'Ġmen',
 'yal',
 'ur',
 'kan',
 'Ġw',
 'ang',
 'Ġdan',
 'Ġmem',
 'ud',
 'ah',
 'kan',
 'Ġum',
 'at',
 'ĠIslam',
 'Ġyang',
 'Ġakan',
 'Ġmen',
 'una',
 'ikan',
 'Ġsol',
 'at',
 'Ġhari',
 'Ġr',
 'aya',
 ',',
 'Ġyang',
 'Ġmungkin',
 'Ġlebih',
 'Ġaw',
 'al',
 'Ġdar',
 'ip',
 'ada',
 'Ġke',
 'put',
 'usan',
 'Ġk',
 'era',
 'ja',
 'an',
 'Ġpada',
 'Ġhari',
 'ĠSab',
 'tu',
 'Ġl',
 'usa',
 '.',
 'Ġes',
 'ok',
 '.',
 '<|28.98|>',
 '<|endoftext|>']

In [27]:
results = mp.multiprocessing(files, loop, cores = 50)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/206 [00:00<?, ?it/s]Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens ha

  4%|▍         | 9/206 [04:32<1:24:09, 25.63s/it]]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
  6%|▌         | 12/206 [05:37<1:34:45, 29.31s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during ge

  7%|▋         | 14/206 [06:50<1:17:39, 24.27s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
 15%|█▌        | 31/206 [06:54<17:19,  5.94s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
  9%|▉         | 19/206 [08:58<1:09:38, 22.35s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure Wh

...

 84%|████████▍ | 174/206 [31:41<06:33, 12.29s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
 85%|████████▌ | 176/206 [32:05<06:04, 12.16s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
 90%|████████▉ | 185/206 [32:26<04:05, 11.70s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure Whi

 87%|████████▋ | 179/206 [32:33<05:22, 11.93s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
 90%|█████████ | 186/206 [32:37<03:46, 11.34s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
 90%|████████▉ | 185/206 [33:07<04:03, 11.57s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure Whi

 95%|█████████▍| 195/206 [33:46<01:37,  8.89s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
 93%|█████████▎| 191/206 [34:03<02:30, 10.06s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during gene

 98%|█████████▊| 201/206 [34:39<00:17,  3.54s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
 94%|█████████▍| 194/206 [34:43<02:10, 10.90s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
 96%|█████████▌| 198/206 [35:21<01:21, 10.14s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure Whi

 98%|█████████▊| 201/206 [35:45<00:43,  8.73s/it]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
100%|██████████| 206/206 [36:00<00:00, 10.49s/it]
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
100%|██████████| 206/206 [36:23<00:00, 10.60s/it]


In [28]:
len(results)

320855

In [29]:
results[0]

{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Itu yang selalu<|3.00|><|3.00|> Itu kadang-kadang<|4.00|><|4.00|> Yang salah kita selalu<|6.82|><|6.82|> Sentiasa ada<|8.92|><|8.92|> Sebab tu saya cakap tadi baru nak pulangkan<|11.14|><|11.14|> Dalam erti kata itu saya bersyukur<|13.00|><|13.00|> Istilahnya apabila saya<|14.92|><|14.92|> Mempunyai contohnya<|16.98|><|16.98|> Suka Instagram atau YouTube atau sesuatu<|18.68|><|18.68|> Ia lebih kepada saya, saya hanya mahu<|20.78|><|20.78|> Ia seperti berkongsi<|23.40|><|23.40|> Kerana jika saya mahu fokus<|25.20|><|25.20|> Hanya untuk konflik dan sebagainya<|26.96|><|26.96|> The masalahnya ialah masalah saya dalam dunia kesihatan<|29.24|><|endoftext|>',
 'audio_filename': 'split-indonesian/0-1-44160000.mp3'}

In [30]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(results, test_size = 200)

In [31]:
len(train), len(test)

(320655, 200)

In [ ]:
import IPython.display as ipd
ipd.Audio(results[0]['audio_filename'])

In [33]:
with open('prepared-indonesian-ms.jsonl', 'w') as fopen:
    for r in tqdm(train):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 320655/320655 [00:01<00:00, 194343.51it/s]


In [34]:
!ls -lh prepared-indonesian-ms.jsonl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


-rw-r--r-- 1 ubuntu ubuntu 189M Apr 25 12:21 prepared-indonesian-ms.jsonl


In [15]:
from huggingface_hub import HfApi
api = HfApi()

In [36]:
api.upload_file(
    path_or_fileobj='prepared-indonesian-ms.jsonl',
    path_in_repo='prepared-indonesian-ms.jsonl',
    repo_id='mesolitica/pseudolabel-indonesian-large-v3-timestamp',
    repo_type='dataset',
)

prepared-indonesian-ms.jsonl:   0%|          | 0.00/198M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/pseudolabel-indonesian-large-v3-timestamp/blob/main/prepared-indonesian-ms.jsonl'

In [37]:
!head -n 10 prepared-indonesian-ms.jsonl

{"new_text": "<|startoftranscript|><|ms|><|transcribe|><|0.00|> Terdapat kira-kira 20 ribu orang kesemuanya yang dikesan di sini, di kawasan konvoi, di Menara Nasdem, dan juga di KPU kesemuanya untuk mengikuti perkembangan konvoi ini.<|14.96|><|16.42|> Dan bilangan ini adalah gabungan kader parti ahli gabungan dan sukarelawan.<|24.28|><|endoftext|>", "audio_filename": "split-indonesian/1-160-118560000.mp3"}
{"new_text": "<|startoftranscript|><|ms|><|transcribe|><|0.00|> Saya tidak akan dibenarkan untuk pergi ke terlalu mendalam maka apakah psikologinya tetapi ini adalah bagaimana saya mengajar<|5.88|><|5.88|> matahari timur yang mulia Saya memulakan buku yang saya tidak bawa juga dalam kes<|12.72|><|12.72|> keganasan seksual di perspektif thymology pertama yang sering berlaku di spaces private<|19.80|><|21.44|> Hai, jadi apa yang pasti untuk dipilih sebagai bukti Satu-satunya bukti yang biasanya dikemukakan oleh pihak pendakwa ialah<|27.72|><|endoftext|>", "audio_filename": "split-indo

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [38]:
with open('test-indonesian-dataset-ms.json', 'w') as fopen:
    json.dump(test, fopen)

In [6]:
import json

with open('test-indonesian-dataset-ms.json') as fopen:
    test = json.load(fopen)

In [39]:
api.upload_file(
    path_or_fileobj='test-indonesian-dataset-ms.json',
    path_in_repo='id-ms/test-indonesian-dataset-ms.json',
    repo_id='mesolitica/speech-test-set',
    repo_type='dataset',
)

test-indonesian-dataset-ms.json:   0%|          | 0.00/125k [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/speech-test-set/blob/main/id-ms/test-indonesian-dataset-ms.json'

In [7]:
!rm -rf id-ms
!mkdir id-ms

In [10]:
import shutil

for t in test:
    f = os.path.split(t['audio_filename'])[1]
    shutil.copyfile(t['audio_filename'], os.path.join('id-ms', f)) 
    
len(glob('id-ms/*'))

100%|██████████| 200/200 [00:01<00:00, 150.00it/s]


200

In [11]:
# !sudo apt install zip -y

In [13]:
# !zip -r test-id-ms-audio.zip id-ms

In [16]:
api.upload_file(
    path_or_fileobj='test-id-ms-audio.zip',
    path_in_repo='id-ms/test-id-ms-audio.zip',
    repo_id='mesolitica/speech-test-set',
    repo_type='dataset',
)

test-id-ms-audio.zip:   0%|          | 0.00/88.7M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/speech-test-set/blob/main/id-ms/test-id-ms-audio.zip'